In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_desc
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
 
AS
WITH 
assortment_sap AS (
    SELECT * EXCEPT(rn) 
    FROM (
        SELECT
            rt_asort AS assortment, 
            txtlg AS name, 
            ROW_NUMBER() OVER (PARTITION BY rt_asort ORDER BY end_date DESC) AS rn 
        FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_trt_asort
    ) 
    WHERE rn = 1
), 
assortment_site_active AS (
    SELECT DISTINCT assortment 
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_site_active 
),
main AS (
    SELECT 
        assortment_sap.assortment,
        name,
        CASE 
            WHEN (name LIKE '1%' OR name LIKE '2%' OR name LIKE '3%') AND assortment_site_active.assortment IS NOT NULL THEN 'ACTIVE'
            ELSE 'INACTIVE'
        END AS status,
        CASE 
            WHEN name LIKE '2%' THEN 'DC_ETON'
            WHEN name LIKE '3%' THEN 'DC_WCM'
            ELSE ''
        END AS asrt_group
    FROM assortment_sap 
    LEFT JOIN assortment_site_active ON assortment_sap.assortment = assortment_site_active.assortment
)
SELECT DISTINCT
    main.assortment as assortment,
    name,
    status,
    asrt_group,
    a1.assortment_class,
    a1.assortment_class_desc,
    CASE
        WHEN UPPER(main.assortment) RLIKE 'KVI|RUOU|THUOCLA' THEN 1
        WHEN UPPER(main.assortment) LIKE '%CAOCAP%' THEN 2
        WHEN UPPER(main.assortment) LIKE '%CORETQ%' THEN 3
        WHEN UPPER(main.assortment) RLIKE 'HANOI|HCM|HN|HCM|CANTHO|HP|_DN|_NA|_TN|_NB|_TH|_BG|_CT|_BDINH|_NTRANG|_DLK|_PYEN|_LS' THEN 4
        WHEN UPPER(main.assortment) RLIKE '_NTB|_MD|_BTB' THEN 7
        WHEN UPPER(main.assortment) RLIKE 'M_B|M_N|MB|MN|MT|N_|B_|_S|T_|_N|_B' THEN 8
        WHEN UPPER(main.assortment) RLIKE 'SOUTH|NORTH|NORT|SOUT' THEN 9
        WHEN UPPER(main.assortment) RLIKE '_TQ' THEN 10
        WHEN NOT RLIKE(UPPER(main.assortment), '^[0-9]+$') THEN 11
        ELSE 12
    END AS priority
FROM main
LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_class a1 ON main.assortment = a1.assortment
""")